# Exploratory data analysis

Data source: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

Important attributes description:
* SalePrice: The property's sale price in dollars. This is the target variable that you're trying to predict.
* MSSubClass: The building class
* BldgType: Type of dwelling
* HouseStyle: Style of dwelling
* OverallQual: Overall material and finish quality
* OverallCond: Overall condition rating
* YearBuilt: Original construction date
* Heating: Type of heating
* CentralAir: Central air conditioning
* GrLivArea: Above grade (ground) living area square feet
* BedroomAbvGr: Number of bedrooms above basement level

## Import used packages

In [ ]:
import pandas as pd # dataframes
import numpy as np # matrices and linear algebra
import matplotlib.pyplot as plt # plotting
import seaborn as sns # another matplotlib interface - styled and easier to use

## Load the data into the Pandas DataFrame - in our case it is a csv file

In [ ]:
# df = pd.read_csv('https://homel.vsb.cz/~pro0199/data_exploration/house_prices_dataset.csv', sep=',')
df = pd.read_csv('house_prices_dataset.csv', sep=',')
df.head()

## Price is the most interisting attribute in our dataset, let's take a look on its distribution

#### Vizualize house prices in form of histogram
- What does it tell us about the prices?

In [ ]:
sns.displot(df.SalePrice)

## Let's add more complexity to histogram vizualizations. Does price change for different values of GaragaFinish attribute?

#### GarageFinish: Interior finish of the garage
- Fin	Finished
- RFn	Rough Finished	
- Unf	Unfinished
- NA	No Garage

In [ ]:
sns.displot(data=df, x='SalePrice', hue='GarageFinish', edgecolor='white')

In [ ]:
df[['GarageFinish', 'SalePrice']].describe(include='all')

### We can see that houses with no garage are somehow missing
- The no garage houses have a *nan* value in the feature so they are not plotted

In [ ]:
df['GarageFinish'] = df['GarageFinish'].fillna('NoGarage')

In [ ]:
sns.displot(data=df, x='SalePrice', hue='GarageFinish', edgecolor='white')

### Another very useful tecnique for the numerical features distributions are boxplots
- Did you see any of these before?
- How should we read them?
    - Quartiles
    - Median
    - Box size
    - Outliers and IQR
    - Min/max - with or without outliers

### Plot only the *SalePrice* using boxplot

In [ ]:
sns.boxplot(data=df, y='SalePrice')

## Let's take a look at the *SalePrice* for different *BldgType* values 
- What can we say about the prices?

In [ ]:
fig = plt.figure(figsize=(16, 9))
sns.boxplot(data=df, y='SalePrice', x='BldgType')

## We have an information about a month and a year for the sold houses.
### Can you vizualize what was the average price of the house by quarters?

* We need to create a new columns with merged Year and Quarter information in this pattern: '2010-Q1','2010-Q2', etc.
* We can map the values using the *apply* function or use string concatenation directly
* If you need to change data type of the columns, you can use *astypy*

In [ ]:
df['QuarterSold'] = df.MoSold // 4  + 1
df['YearQuarterSold'] = df.YrSold.astype(str) + '-' + df.QuarterSold.astype(str)
df['YearQuarterSold'].head()

In [ ]:
df['YearQuarterSoldApply'] = df.loc[:, ['MoSold', 'YrSold']].apply(lambda x: f'{x[1]}-{(x[0]//4 + 1)}', axis=1)
df['YearQuarterSoldApply'].head()

### Compute the average price for the *YearQuarterSold* attribute

In [ ]:
df_agg = df.groupby('YearQuarterSold').SalePrice.mean().reset_index(name='MeanPrice')
df_agg.head()

## Vizualize the data using *lineplot*
- In case of xlabels mixing try to tune the *rotation* and *horizontalalignment* parameters

In [ ]:
fig = plt.figure(figsize=(9,6))
sns.lineplot(data=df_agg, x='YearQuarterSold', y='MeanPrice')
plt.xticks(rotation=65, horizontalalignment='right')
plt.show()

### Add the max, min and median to the plot
- Use *describe* and the [Pandas Melt](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html) function
    - use YearQuarterSold as an *id_vars*
    - use 'min', 'mean', '50%', 'max' as *value_vars*

In [ ]:
df.groupby('YearQuarterSold').SalePrice.describe().reset_index()

In [ ]:
df_agg = df.groupby('YearQuarterSold').SalePrice.describe().reset_index()
df_melt = pd.melt(df_agg, id_vars=['YearQuarterSold'], value_vars=['min', 'mean', '50%', 'max'])
df_melt

## What can we say about the minimum and maximum values?
- hint - std. deviation? outliers?

In [ ]:
fig = plt.figure(figsize=(9,6))
sns.lineplot(data=df_melt, x='YearQuarterSold', y='value', hue='variable')
plt.xticks(rotation=65, horizontalalignment='right')
plt.show()

## Combining boxplots with calculated mean value

In [ ]:
fig = plt.figure(figsize=(9,6))
sns.boxplot(data=df, x='YearQuarterSold', y='SalePrice', color='grey', order=sorted(df.YearQuarterSold.unique()))
sns.lineplot(data=df_agg, x='YearQuarterSold', y='mean', color='red')
plt.xticks(rotation=65, horizontalalignment='right')
plt.show()

# We can take a look at number of sold houses for the defined time periods as well
- We will use standard bar plot
- In which quarter were the most houses sold?
- Fun facts: [https://themortgagereports.com/44135/whats-the-best-time-of-year-to-sell-a-home](https://themortgagereports.com/44135/whats-the-best-time-of-year-to-sell-a-home)

In [ ]:
df_cnt = df.groupby('YearQuarterSold').SalePrice.count().reset_index(name='Count')
df_cnt

In [ ]:
fig = plt.figure(figsize=(9,6))
sns.barplot(data=df_cnt, x='YearQuarterSold', y='Count')
plt.xticks(rotation=65, horizontalalignment='right')
plt.show()

# Pairplot
- Plot pairwise relationships in a dataset
- Scatterplots for multiple pairs of numerical attributes, see [this](https://seaborn.pydata.org/generated/seaborn.pairplot.html)
- Select only numerical attributes from dataframe - *select_dtypes*

In [ ]:
df.select_dtypes(include=np.number)

#### If the dataset has too many columns, it is good to set pandas parameter *display.max_columns* to None so there is no such limit

In [ ]:
with pd.option_context('display.max_columns', None):
  display(df.select_dtypes(include=np.number))

#### Select only an interesting features for paiplot visualization
- The pairplot might be too big to see anything in the visualization

In [ ]:
selected_columns = ['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'GrLivArea', 'GarageYrBlt', 'GarageArea', 'SalePrice']
sns.pairplot(df[selected_columns])

#### Notice that both OverallQual and OverallCond columns are rather categorical features even though they are represented by numerical values thus one would expect that the features are numerical but they aren't so scatter plot may not be the best way to visualize this type of data

- Let's select only appropropriate features this time

In [ ]:
selected_columns = ['LotFrontage', 'LotArea', 'YearBuilt', 'GrLivArea', 'GarageYrBlt', 'GarageArea', 'SalePrice']
sns.pairplot(data=df, vars=selected_columns, hue='BldgType')

## Use swarmplot/striplot for displaying OverallQual, OverallCond as categorical-numerical relationship

In [ ]:
sns.swarmplot(data=df, x='OverallQual', y='SalePrice')

In [ ]:
sns.stripplot(data=df, x='OverallQual', y='SalePrice')

In [ ]:
sns.stripplot(data=df, x='OverallQual', y='SalePrice', jitter=0.4)

In [ ]:
plt.figure(figsize=(16,5))
sns.swarmplot(data=df, x='OverallQual', y='SalePrice', size=2)

# Are you able to say which categorical attributes effect the price right now?
- Yeah... me neither so let's find out by plotting the data

## Start with selection of only the categorical features

In [ ]:
qualitative_columns = df.select_dtypes(exclude=np.number).columns
for c in qualitative_columns:
    if df[c].isnull().any():        
        df[c] = df[c].fillna('MISSING')
    
qualitative_columns

In [ ]:
df.head()

### Now we need to [melt](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html) the dataset into the standard data format

In [ ]:
f = pd.melt(df, id_vars=['SalePrice'], value_vars=qualitative_columns)
f.head()

### And finally we can use the [FacetGrid](https://seaborn.pydata.org/generated/seaborn.FacetGrid.html) to crate a grid of subplots and map boxplot to every cell

In [ ]:
def boxplot(x, y, **kwargs):
    sns.boxplot(x=x, y=y)
    x=plt.xticks(rotation=90)

g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, height=5)
g = g.map(boxplot, "value", "SalePrice")

## Another very useful feature is the [subplots](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html) function for defining our own grids
### We will plot boxplots together with the individual value counts using barplot for every categorical feature

In [ ]:
fig, axes = plt.subplots(len(qualitative_columns), 2, figsize=(10, 100))

for i, col in enumerate(qualitative_columns):
    ax_left = fig.axes[2*i]
    ax_right = fig.axes[2*i + 1]
    sns.boxplot(data=df, x=col, y='SalePrice', ax=ax_left)
    ax_left.set_xticklabels(ax_left.xaxis.get_majorticklabels(), rotation=60)
    sns.countplot(data=df, x=col, ax=ax_right)
    ax_right.set_xticklabels(ax_right.xaxis.get_majorticklabels(), rotation=60)
        
fig.tight_layout()

# Categorical heatmap
### Let's vizualize a relationship between *HouseStyle* and *Neighborhood* using heatmap with counts of every combination of the two attributes

In [ ]:
df_agg = df.groupby(['HouseStyle', 'Neighborhood']).Id.count().reset_index(name='count')
df_agg

In [ ]:
plt.figure(figsize=(15, 6))
sns.heatmap(data=df_agg.pivot(index='HouseStyle', columns='Neighborhood', values='count'), cmap='Greens')
plt.show()

### Let's vizualize a relationship between *HouseStyle* and *BsmtFinType1* using heatmap with counts of every combination of the two attributes

In [ ]:
df_agg = df.groupby(['HouseStyle', 'BsmtFinType1']).Id.count().reset_index(name='count')
df_agg.head()

**HouseStyle: Style of dwelling**

       1Story	One story
       1.5Fin	One and one-half story: 2nd level finished
       1.5Unf	One and one-half story: 2nd level unfinished
       2Story	Two story
       2.5Fin	Two and one-half story: 2nd level finished
       2.5Unf	Two and one-half story: 2nd level unfinished
       SFoyer	Split Foyer
       SLvl	Split Level

**BsmtFinType1: Rating of basement finished area**

       GLQ	Good Living Quarters
       ALQ	Average Living Quarters
       BLQ	Below Average Living Quarters	
       Rec	Average Rec Room
       LwQ	Low Quality
       Unf	Unfinshed
       NA	No Basement

In [ ]:
sns.heatmap(data=df_agg.pivot(index='HouseStyle', columns='BsmtFinType1', values='count'), cmap='Greens')

# Correlation

Resources containing refreshing informations about correlations:
- Take a look at [this link](https://www.dummies.com/education/math/statistics/how-to-interpret-a-correlation-coefficient-r/)
- See also [this](https://www.simplypsychology.org/correlation.html) or [this](https://www.investopedia.com/ask/answers/032515/what-does-it-mean-if-correlation-coefficient-positive-negative-or-zero.asp) for some more info about the topic

In [ ]:
corr_mat = df.corr()
corr_mat

In [ ]:
plt.figure(figsize=(12, 9))
sns.heatmap(corr_mat)

## Which color will be more appropropriate for corellation representation?
- Change color map *cmap* (e.g. RdYlGn)
- Set limits to adjust color map values from -1 to 1 (attributes *vmin*, *vmax*)

In [ ]:
plt.figure(figsize=(12, 9))
sns.heatmap(corr_mat, square=True, cmap='RdYlGn', vmin=-1, vmax=1)

## What can we say about the values in this correlation sub-matrix?
- Let's focus only some subset of the features

In [ ]:
selected_columns = ['LotFrontage', 'LotArea', 'YearBuilt', 'GrLivArea', 'GarageYrBlt', 'GarageArea', 'SalePrice']
corr_mat = df[selected_columns].corr()
corr_mat

In [ ]:
plt.figure(figsize=(12, 9))
sns.heatmap(corr_mat, square=True, cmap='RdYlGn', vmin=-1, vmax=1)

## Add values to the heatmap and amplify heatmap's readability

In [ ]:
plt.figure(figsize=(12, 9))
sns.heatmap(corr_mat, square=True, cmap=sns.light_palette("seagreen", as_cmap=True), vmin=0.0, annot=True)

## We highly recommend taking a look at the plot gallery for some inspiration!
- https://seaborn.pydata.org/api.html

# Individual work

## Find the oldest sold house for each locality (attribute **Neighborhood**).

## Explore the meaning of values in attribute **LotConfig**. Is there any relationship between **LotConfig** and **LotFrontage**, create visualization and describe it.

## Do various types of homes differ in living area? Explore the effect of **HouseStyle** and **GrLivArea**. Create visualization and describe it.

## Does **BldfType** have effect on **GrLivArea**? Create visualization and describe it.

## How are the **HouseStyle** and **BldgType** values combined? What is the most common combination? Which feature type are these two attributes and what type of visualization can be used to show number of houses in all combinations between values in these two attributes?

## Is there any progress in type of foundation over the years? Take a look to attributes **Fondation** and **YearBuilt**, create visualization and describe it.

## Show me your creativity :-). Come up with one more analytical question, create visualization and interpret results.